In [31]:
!pip freeze | grep pandas

pandas==1.4.2


In [32]:
import pickle
import pandas as pd

In [34]:
with open('../model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/Users/m.shark/anaconda3/envs/venv_mlops/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/m.shark/anaconda3/envs/venv_mlops/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [36]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [37]:
df = read_data('/Users/m.shark/Documents/mlops/mlops-zoomcamp/04-deployment/data/fhv_tripdata_2021-02.parquet')

In [38]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [39]:
print("MEAN predicted duration for Feb 2021 - {}". format(round(y_pred.mean(), 2)))
df['ride_id'] = '{:04d}/{:02d}_'.format(2021, 2) + df.index.astype('str')
df['prediction'] = y_pred
df[['ride_id', 'prediction']].to_parquet(
'prediction',
    engine='pyarrow',
    compression=None,
    index=False
)
print(f"Size of output file {os.path.getsize('prediction')/(1024*1024)} MB")

MEAN predicted duration for Feb 2021 - 16.19
Size of output file 18.79828929901123 MB


In [30]:
email = 'shark.margaret@gmail.com'
from hashlib import sha1
def compute_hash(email):
    return sha1(email.lower().encode('utf-8')).hexdigest()
compute_hash(email)

'bfa98b49576486fd6e4f3b1d3eb5603d573b67fe'